<a href="https://colab.research.google.com/github/toche7/DataAnalytic/blob/main/Lab02Outlier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Lab: การตรวจจับและจัดการ Outlier

## 1. Import Libraries & Load Data

ในส่วนนี้ เราจะนำเข้าไลบรารีที่จำเป็นสำหรับการวิเคราะห์ข้อมูลและโหลดชุดข้อมูลที่เราจะใช้

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# โหลด DataFrame ของคุณที่นี่ ตัวอย่าง:
# df = pd.read_csv('your_data.csv')
# หรือสร้าง DataFrame ตัวอย่าง:
data = {'col1': np.random.randn(100) * 10 + 50,
        'col2': np.random.rand(100) * 100,
        'col3': np.random.randn(100) * 5 + 20}
df = pd.DataFrame(data)

# เพิ่ม outlier เข้าไปใน DataFrame ตัวอย่าง
df.loc[0, 'col1'] = 150
df.loc[1, 'col2'] = -200
df.loc[2, 'col3'] = 100

display(df.head())

## 2. นิยาม Outlier และผลกระทบ

Outlier (ค่าผิดปกติ) คือค่าข้อมูลที่อยู่ห่างไกลจากค่าส่วนใหญ่ในชุดข้อมูลอย่างมีนัยสำคัญ

**ทำไมต้องตรวจจับและจัดการ Outlier?**

*   **ส่งผลกระทบต่อค่าสถิติ:** Outlier สามารถทำให้ค่าเฉลี่ย (mean) และค่าเบี่ยงเบนมาตรฐาน (standard deviation) ผิดเพี้ยนไปได้
*   **ส่งผลต่อโมเดล:** ในการสร้างแบบจำลองทางสถิติหรือ machine learning, outlier สามารถทำให้แบบจำลองมีความแม่นยำลดลง หรือทำให้ผลลัพธ์มีความเอนเอียง (biased)
*   **บ่งชี้ข้อผิดพลาด:** Outlier อาจเป็นสัญญาณของข้อผิดพลาดในการบันทึกข้อมูล การวัด หรือกระบวนการเก็บรวบรวมข้อมูล
*   **ข้อมูลเชิงลึก:** ในบางกรณี Outlier อาจเป็นข้อมูลที่มีความสำคัญและบ่งชี้ถึงเหตุการณ์หรือปรากฏการณ์ที่ผิดปกติ ซึ่งอาจเป็นประโยชน์ในการวิเคราะห์

## 3. IQR Method

วิธี IQR (Interquartile Range) เป็นวิธีที่ใช้กันทั่วไปในการตรวจจับ Outlier โดยอาศัยการคำนวณค่าควอไทล์

*   **Q1 (First Quartile):** ค่าที่แบ่งข้อมูล 25% แรกออกจากข้อมูลที่เหลือ
*   **Q3 (Third Quartile):** ค่าที่แบ่งข้อมูล 75% แรกออกจากข้อมูลที่เหลือ
*   **IQR (Interquartile Range):** Q3 - Q1

เราจะกำหนดขอบเขตสำหรับค่าปกติคือ `[Q1 - 1.5 * IQR, Q3 + 1.5 * IQR]` ค่าใดๆ ที่อยู่นอกขอบเขตนี้จะถือว่าเป็น Outlier

In [ ]:
# ตัวอย่างโค้ดสำหรับ IQR Method
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# ลองตรวจจับ outlier ในคอลัมน์ 'col1'
outliers_col1_iqr = detect_outliers_iqr(df, 'col1')
print(f"Outliers ใน col1 (IQR Method):\n{outliers_col1_iqr}")

## 4. Z-score Method

วิธี Z-score เป็นอีกวิธีหนึ่งในการตรวจจับ Outlier โดยอาศัยค่าเฉลี่ยและค่าเบี่ยงเบนมาตรฐาน

Z-score ของจุดข้อมูลหนึ่งๆ คือจำนวนเท่าของค่าเบี่ยงเบนมาตรฐานที่จุดข้อมูลนั้นอยู่ห่างจากค่าเฉลี่ย

$$ Z = \frac{x - \mu}{\sigma} $$

โดยที่:
*   $x$ คือจุดข้อมูล
*   $\mu$ คือค่าเฉลี่ยของชุดข้อมูล
*   $\sigma$ คือค่าเบี่ยงเบนมาตรฐานของชุดข้อมูล

โดยทั่วไป หากค่า Z-score มีค่าสัมบูรณ์มากกว่า 3 (|Z| > 3) จุดข้อมูลนั้นจะถูกพิจารณาว่าเป็น Outlier

In [ ]:
# ตัวอย่างโค้ดสำหรับ Z-score Method
from scipy.stats import zscore

def detect_outliers_zscore(df, column, threshold=3):
    z_scores = np.abs(zscore(df[column]))
    outliers = df[z_scores > threshold]
    return outliers

# ลองตรวจจับ outlier ในคอลัมน์ 'col1'
outliers_col1_zscore = detect_outliers_zscore(df, 'col1')
print(f"Outliers ใน col1 (Z-score Method):\n{outliers_col1_zscore}")

## 5. Visualization

การแสดงภาพเป็นวิธีที่มีประสิทธิภาพในการตรวจจับ Outlier

*   **Boxplot:** แสดงการกระจายของข้อมูลและระบุ Outlier ด้วยจุดที่อยู่นอก "หนวด" (whiskers) ของกล่อง
*   **Scatter plot:** สามารถใช้เพื่อดูความสัมพันธ์ระหว่างตัวแปรและระบุจุดข้อมูลที่อยู่ห่างจากแนวโน้มทั่วไป

In [ ]:
# ตัวอย่างโค้ดสำหรับการแสดงภาพ
# Boxplot
plt.figure(figsize=(10, 5))
sns.boxplot(x=df['col1'])
plt.title('Boxplot of col1')
plt.show()

# Scatter plot
plt.figure(figsize=(10, 5))
plt.scatter(df.index, df['col1'])
plt.title('Scatter Plot of col1')
plt.xlabel('Index')
plt.ylabel('col1')
plt.show()

## 6. Handling Outliers

เมื่อตรวจพบ Outlier แล้ว มีหลายวิธีในการจัดการกับมัน ขึ้นอยู่กับลักษณะของข้อมูลและวัตถุประสงค์ของการวิเคราะห์

*   **Remove (ลบ):** ลบแถวที่มี Outlier ออกจาก DataFrame วิธีนี้ควรใช้ด้วยความระมัดระวัง โดยเฉพาะอย่างยิ่งหากมี Outlier จำนวนมาก
*   **Cap (Winsorize):** แทนค่า Outlier ด้วยค่าขอบเขตที่กำหนด เช่น แทนค่าที่มากกว่าขอบเขตบนด้วยค่าขอบเขตบน และค่าที่น้อยกว่าขอบเขตล่างด้วยค่าขอบเขตล่าง (มักใช้ Q1 - 1.5*IQR และ Q3 + 1.5*IQR เป็นขอบเขต)
*   **Transform (แปลงข้อมูล):** ใช้ฟังก์ชันทางคณิตศาสตร์เพื่อลดผลกระทบของ Outlier เช่น Log transformation หรือ Box–Cox transformation

In [ ]:
# ตัวอย่างโค้ดสำหรับการจัดการ Outlier (Cap/Winsorize)
def cap_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_capped = df.copy()
    df_capped[column] = np.where(df_capped[column] < lower_bound, lower_bound, df_capped[column])
    df_capped[column] = np.where(df_capped[column] > upper_bound, upper_bound, df_capped[column])
    return df_capped

# ลอง Cap Outlier ในคอลัมน์ 'col1'
df_capped_col1 = cap_outliers_iqr(df.copy(), 'col1')
print("DataFrame หลัง Cap Outlier ใน col1:")
display(df_capped_col1.head())

In [ ]:
# ตัวอย่างโค้ดสำหรับการจัดการ Outlier (Remove)
def remove_outliers_iqr(df, column):
     Q1 = df[column].quantile(0.25)
     Q3 = df[column].quantile(0.75)
     IQR = Q3 - Q1

     lower_bound = Q1 - 1.5 * IQR
     upper_bound = Q3 + 1.5 * IQR

     df_cleaned = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
     return df_cleaned

# ลอง Remove Outlier ในคอลัมน์ 'col1'
df_cleaned_col1 = remove_outliers_iqr(df.copy(), 'col1')
print("\nDataFrame หลัง Remove Outlier ใน col1:")
display(df_cleaned_col1.head())

## 7. เปรียบเทียบผลก่อน–หลัง

หลังจากจัดการ Outlier แล้ว ควรเปรียบเทียบผลลัพธ์ก่อนและหลังการจัดการเพื่อดูว่าการจัดการมีผลอย่างไรต่อการกระจายของข้อมูลและค่าสถิติ

*   **Descriptive Statistics:** ใช้ `df.describe()` เพื่อดูค่าสถิติสรุป เช่น ค่าเฉลี่ย ค่ามัธยฐาน ค่าเบี่ยงเบนมาตรฐาน ก่อนและหลังการจัดการ Outlier
*   **Histogram:** วาด histogram เพื่อดูการกระจายของข้อมูลก่อนและหลังการจัดการ Outlier

In [ ]:
# ตัวอย่างโค้ดสำหรับการเปรียบเทียบผลก่อน-หลัง
# เปรียบเทียบ Descriptive Statistics
print("Descriptive Statistics ก่อนจัดการ Outlier:")
display(df['col1'].describe())

print("\nDescriptive Statistics หลัง Cap Outlier ใน col1:")
display(df_capped_col1['col1'].describe())

print("\nDescriptive Statistics หลัง Remove Outlier ใน col1:")
display(df_cleaned_col1['col1'].describe())

# เปรียบเทียบ Histogram
plt.figure(figsize=(12, 5))

plt.subplot(1, 3, 1)
sns.histplot(df['col1'], kde=True)
plt.title('Histogram before Outlier')

plt.subplot(1, 3, 2)
sns.histplot(df_capped_col1['col1'], kde=True)
plt.title('Histogram after Cap Outlier')

plt.subplot(1, 3, 3)
sns.histplot(df_cleaned_col1['col1'], kde=True)
plt.title('Histogram after Remove Outlier')

plt.tight_layout()
plt.show()

## 8. Hands-on Exercise

ให้นักเรียนเลือกตัวแปร 2–3 ตัวจากชุดข้อมูลของคุณ (หรือใช้ DataFrame ตัวอย่างข้างต้น) แล้วทำตามขั้นตอนต่อไปนี้:

1.  **เลือกตัวแปร:** ระบุชื่อคอลัมน์ที่คุณต้องการวิเคราะห์ Outlier
2.  **ตรวจจับ Outlier:** ใช้ทั้งวิธี IQR และ Z-score เพื่อตรวจจับ Outlier ในตัวแปรที่เลือก
3.  **แสดงภาพ:** ใช้ Boxplot และ Scatter plot เพื่อแสดง Outlier
4.  **จัดการ Outlier:** เลือกวิธีจัดการ Outlier ที่เหมาะสม (เช่น Remove หรือ Cap) และนำไปใช้กับตัวแปรที่เลือก
5.  **เปรียบเทียบผล:** เปรียบเทียบค่าสถิติสรุปและ Histogram ของตัวแปรที่เลือกก่อนและหลังการจัดการ Outlier
6.  **สรุปผล:** เขียนสรุปใน markdown cell อธิบายว่าคุณทำอะไร ผลลัพธ์ที่ได้เป็นอย่างไร และวิธีจัดการ Outlier ที่คุณเลือกมีผลอย่างไรต่อข้อมูล

### สรุป
ความสำคัญของ Outlier Detection

- **ป้องกันข้อมูลบิดเบือน (Bias) และผลลัพธ์ผิดพลาด**  
  - Outlier อาจเกิดจากการกรอกข้อมูลผิด ข้อผิดพลาดในการวัด หรือสถานการณ์พิเศษ  
  - หากไม่จัดการ อาจลากค่า mean, variance และค่าสถิติอื่น ๆ ให้บิดเบือน  

- **ปรับปรุงคุณภาพข้อมูลก่อน Modeling**  
  - การลบหรือแก้ไข outlier ช่วยให้โมเดลเรียนรู้ pattern หลักของข้อมูลได้ดีขึ้น  
  - ลดโอกาส overfitting ที่อาจเกิดจาก noise  

- **เลือกวิธีจัดการที่เหมาะสมกับงาน**  
  - **Remove**: ลบข้อมูลที่ไม่เป็นประโยชน์  
  - **Cap (Winsorize)**: แทนค่าด้วยขอบเขต IQR  
  - **Transform**: ปรับ distribution ด้วย log/Box–Cox  

- **เสริมความเข้าใจผ่าน Visualization**  
  - Boxplot และ scatter plot ช่วยให้มองเห็น outlier ชัดเจน  
  - สนับสนุนการตัดสินใจว่าจะรักษาหรือกำจัดจุดผิดปกติ  

- **สร้างมาตรฐานในขั้นตอน EDA**  
  - หลีกเลี่ยงการมองข้ามข้อมูลสำคัญหรือมองข้าม noise  
  - ทำให้ pipeline การวิเคราะห์มีความน่าเชื่อถือและ reproducible  